# IMPORTS DE LIBRAIRIES

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
import torch 
print("torch version           : ", torch.__version__)
print("torch cuda version      : ", torch.version.cuda)
print("torch.cuda.is_available : ", torch.cuda.is_available())

In [ ]:
import detectron2
print("detectron2 version : ", detectron2.__version__)

In [ ]:
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.config import get_cfg, get_stack_cell_config
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data.datasets import get_dicts
from detectron2.modeling import build_model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json, cv2, random, glob


In [ ]:
from detectron2.utils.logger import setup_logger
setup_logger()

In [ ]:
def imBGRshow(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
def imRGBshow(img):
    plt.imshow(img)
    plt.show()

# REGISTER LES IMAGES
## /!\ CHANGE THE DATA PATH ACCORDINGLY

NB : Les classes sont les suivantes :
- 0 : Cellule intacte et nette   (Intact_Sharp)
- 1 : Cellule intacte et floue   (Intact_Blurry)
- 2 : Cellule explosée et nette  (Broken_Sharp)
- 3 : Cellule explosée et floue  (Broken_Blurry)

Pour seulement considérer les cellules nettes, utiliser :
classes = {'Intact_Sharp':0, 'Broken_Sharp':2}

Pour considérer tous les types de cellules, utiliser :
classes = {'Intact_Sharp':0,'Intact_Blurry':1,'Broken_Sharp':2,'Broken_Blurry':3}

In [ ]:
classes = {'Intact_Sharp':0, 'Broken_Sharp':2}
#classes = {'Intact_Sharp':0,'Intact_Blurry':1,'Broken_Sharp':2,'Broken_Blurry':3}

## DATASET
De combien de images ? soit combien de stacks?
Le dataset est séparé en 3 jeux de données : 
- 60%    => Entraînement
- 20%    => Validation
- 20%    => Test  


Les données doivent être rangées dans la structure suivante de fichiers. La variable data_path définie dans la variable suivante doit indiquer l'emplacement du dossier Cross-val.  
/!\ ATTENTION, ce chemin est à adapter.  
└── Cross-val  
&emsp;&emsp;&emsp;   ├── Xval0  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   ├── Xval1  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   ├── Xval2  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   ├── Xval3  
&emsp;&emsp;&emsp; |&emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; |&emsp;&emsp;   └── labels  
&emsp;&emsp;&emsp;   └── Xval4  
&emsp;&emsp;&emsp; &emsp;&emsp;   ├── images  
&emsp;&emsp;&emsp; &emsp;&emsp;   └── labels  

Comme son nom l'indique, cette séparation est réalisée afin de pouvoir faire de la validation croisée (cross-validation). Pour des raisons écologiques et de durée d'entraînement, nous n'avons pas tiré profit de cette possibilité, mais il est important de noter qu'elle est facilement implémetable au besoin.  
Un indice indique quelles parties du dataset seront associées avec quel jeu de données (entraînement, validation ou test). Pour réaliser de la validation croisée, il faudra réaliser l'entrainement pour des indices variant de 0 à 4.

In [ ]:
data_path = '/projects/INSA-Image/B01/Data/'
cross_val_idx = 0

In [ ]:
# Modes must have the correct string associated in order to perform the proper operation
mode_train = 'train'
mode_valid = 'val'
mode_test  = 'test'

# By default in our architecture. To use custom names, an override of these names must happen during the configuration (see next section)
dataset_name_train = 'train'
dataset_name_valid = 'val'
dataset_name_test  = 'test'

In [ ]:
# Register the datasets
DatasetCatalog.register(dataset_name_train, lambda: get_dicts(data_path, mode_train, cross_val_idx, classes, dataset_name_train))
DatasetCatalog.register(dataset_name_valid, lambda: get_dicts(data_path, mode_valid, cross_val_idx, classes, dataset_name_valid))
DatasetCatalog.register(dataset_name_test,  lambda: get_dicts(data_path, mode_test,  cross_val_idx, classes, dataset_name_test))

# AFFICHAGE DE QUELQUES IMAGES AVEC LEUR SEGMENTATION MANUELLE

In [ ]:
valid_metadata = MetadataCatalog.get(dataset_name_valid)
valid_dataset_dicts = DatasetCatalog.get(dataset_name_valid)

La cellule suivante permet par défaut d'afficher 2 images ainsi que leur segmentation. Ces 2 images sont tirées au hasard dans le dataset de validation.  
  
Remarque : Si les seules classes représentées sont les cellules nettes, il est très peu probable que des segmentations soient effectuées sur les images tirées au hasard. Pour visualiser des segmentations, il faut soit augmenter le nombre d'images affichées (N) ou alors exécuter plusieurs fois la cellule.

In [ ]:
# Visualize N random samples
N = 2
for data in random.sample(valid_dataset_dicts, N):
    img = cv2.imread(data["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=valid_metadata, scale=1)
    out = visualizer.draw_dataset_dict(data)
    imRGBshow(out.get_image())
    # print(data["file_name"]) # Print the file path

# CONFIG DU RESEAU
Cette configuration est similaire que pour des utilisations de detectron2 normales.  
Il faut cependant changer les configurations par défaut suivantes :
- Architecture
- Input chargé par le dataloader
- Nombre de classes
- Poids du réseaux et couches figées
- Solveur


In [ ]:
config_architecture_file = '../configs/Segmentation-Z/mask_rcnn_z_50.yaml'

# Pour sauvegarder des données d'entrapinement, notamment les poids du réseau entraîné
output_directory = "/local/esaintan/outputs/0/"

In [ ]:
num_gpus = 1      # torch.cuda.device_count()

In [ ]:
# Configuration de base présente dans ../detectron2/config/defaults.py
cfg = get_cfg()

# Configuration de l'architecture (depuis le fichier de configuration défini dans config_architecture_file)
cfg.merge_from_file(config_architecture_file)

# Configuration de l'input : pile. Pour d'autres configuration de pile, il faut soit override les paramètres particuliers configurés dans la fonction, soit écrire une autre fonction.
cfg = get_stack_cell_config(cfg)

# Configuration du nombre de classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)

# Configuration des poids du réseau et des couches figées
cfg.MODEL.BACKBONE.FREEZE_AT = 0      # 0  => aucune couche figée
cfg.MODEL.WEIGHTS = ""                # "" => pas de poids préchargés, ils seront tirés au hasard

# Configuration du solveur
cfg.SOLVER.IMS_PER_BATCH = 1          # Attention à la taille de la mémoire dont dispose la GPU, doit aussi être un multiple du nombre de GPU
if not (cfg.SOLVER.IMS_PER_BATCH % num_gpus):    # Pour être sûr d'être divisible par le nombre de GPU
    cfg.SOLVER.IMS_PER_BATCH = (cfg.SOLVER.IMS_PER_BATCH // num_gpus) * num_gpus
cfg.SOLVER.MAX_ITER = 100      # Pour test, remettre 10000
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.SOLVER.BASE_LR = 0.001

# Configuration du dossier pour sauvegarder les sorties de l'algorithme
cfg.OUTPUT_DIR = output_directory

# La batch norm  Pas frozen sinon ne s'entraîne pas sur la batch norm Options: FrozenBN, GN, "SyncBN", "BN"
cfg.MODEL.RESNETS.NORM = "BN"

# Configuration multi GPU
cfg.SOLVER.REFERENCE_WORLD_SIZE = num_gpus


# La configuration ne pourra plus être modifiée :
cfg.freeze()

# ENTRAINEMENT

In [ ]:
def train():
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    # False to begin training from scratch, 
    # True, takes the specified weights in config, or begin from scratch if no weight specified
    # In our case, since we didn't specify weights, trianing will begin from scratch
    return trainer.train()

launch(train, num_gpus, num_machines=1, dist_url="auto")

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output_directory

# TEST DU MODELE

In [20]:
cfg.defrost()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

In [21]:
predictor = DefaultPredictor(cfg)

/local/esaintan/.conda/envs/PTI/lib/python3.11/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location

In [23]:
test_metadata = MetadataCatalog.get(dataset_name_test)
test_dataset_dicts = DatasetCatalog.get(dataset_name_test)

cross validation 0, mode test: 15444it [00:00, 49214.63it/s]


In [25]:
# Faire visualisation pour une stack

# Visualize N random samples
N = 2
for data in random.sample(test_dataset_dicts, N):
    img = cv2.imread(data["file_name"])
    outputs = predictor(img)
    visualizer = Visualizer(img[:, :, ::-1], metadata=test_metadata, scale=1)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    # out = visualizer.draw_dataset_dict(data)
    imRGBshow(out.get_image())
    # print(data["file_name"]) # Print the file path

ValueError: axes don't match array